In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft

In [3]:
def RECURSIVE_FFT(A,m,w):
    if m==1:
        return A
    else:
        A_even = A[::2]
        A_odd = A[1::2]
        F_even = RECURSIVE_FFT(A_even,m/2,w**2)
        F_odd = RECURSIVE_FFT(A_odd,m/2,w**2)
        F = [0]*int(m)
        x = 1
        for j in range (0,int(m/2)):
            F[j] = F_even[j] + x*F_odd[j]
            F[int(j+m/2)] = F_even[j] - x*F_odd[j]
            x = x * w
        return F

In [4]:
def RECURSIVE_CONVOLUTION(A,B):
    if (len(A)!=len(B)) or np.log2(len(A))%1 > 0:
        raise ValueError("Both coefficient representations must have same size and be power of 2")
    
    # Double degree bound
    n = len(A)
    for i in range (0,n):
        A.append(0)
        B.append(0)
    
    n = len(A)
    wn = np.exp((-2j*np.pi)/n)
    
    # Evaluate using FFT
    FFTA = RECURSIVE_FFT(A,n,wn)
    FFTB = RECURSIVE_FFT(B,n,wn)
    
    # Point-Wise Multiplication
    FC = []
    for i in range (0,n):
        FC.append(FFTA[i]*FFTB[i])
        
    # Interpolate
    F = RECURSIVE_FFT(FC,n,wn**(-1))
    for i in range (0,n):
        F[i] = F[i]*(1/n)  
    
    return F

In [5]:
A = [3,2,1,1]
B = [5,0,2,1]
S2 = np.polymul(A,B)
S1 = RECURSIVE_CONVOLUTION(A,B)

In [6]:
np.allclose(S1[0:len(S2)],S2,rtol=1e-05,atol=1e-08,equal_nan=False)

True

In [7]:
S1

[(15-8.604228440844963e-16j),
 (10-6.106226635438361e-16j),
 (11-1.582067810090848e-15j),
 (11.999999999999998-9.43689570931383e-16j),
 (4+9.159339953157541e-16j),
 (2.9999999999999996+7.216449660063518e-16j),
 (1+1.5265566588595902e-15j),
 (1.7763568394002505e-15+8.326672684688674e-16j)]

In [8]:
S2

array([15, 10, 11, 12,  4,  3,  1])

In [9]:
def REV(n,bits):
    result = 0
    for i in range(int(np.log2(bits))):
        result <<= 1
        result |= n & 1
        n >>= 1
    return result

In [10]:
def BIT_REVERSE_COPY(a,A):
    n = len(a)
    k = 0
    while k<=n-1:
        A[REV(k,n)] = a[k]
        k+=1

In [11]:
def ITERATIVE_FFT(a):
    A = [0]*len(a)
    BIT_REVERSE_COPY(a,A)
    n = len(a)
    
    s = 1
    while s <= np.log2(n):
        m = pow(2,s)
        wm = np.exp((-2j*np.pi)/m)
        k = 0
        while k <= n-1:
            w = 1
            j = 0
            while j <= m/2-1:
                t = w*A[int(k+j+m/2)]
                u = A[int(k+j)]
                A[int(k+j)] = u+t
                A[int(k+j+m/2)] = u-t
                w = w*wm
                j+=1
            k+=m
        s+=1
    return A

In [12]:
def ITERATIVE_INVERSE_FFT(a):
    A = [0]*len(a)
    BIT_REVERSE_COPY(a,A)
    n = len(a)
    
    s = 1
    while s <= np.log2(n):
        m = pow(2,s)
        wm = np.exp((2j*np.pi)/m)
        k = 0
        while k <= n-1:
            w = 1
            j = 0
            while j <= m/2-1:
                t = w*A[int(k+j+m/2)]
                u = A[int(k+j)]
                A[int(k+j)] = u+t
                A[int(k+j+m/2)] = u-t
                w = w*wm
                j+=1
            k+=m
        s+=1
    return A

In [13]:
def ITERATIVE_CONVOLUTION(A,B):
    if (len(A)!=len(B)) or np.log2(len(A))%1 > 0:
        raise ValueError("Both coefficient representations must have same size and be power of 2")
    
    # Double degree bound
    n = len(A)
    for i in range (0,n):
        A.append(0)
        B.append(0)
    
    # Evaluate using FFT
    FFTA = ITERATIVE_FFT(A)
    FFTB = ITERATIVE_FFT(B)
    
    n = len(A)
    
    # Point-Wise Multiplication
    FC = []
    for i in range (0,n):
        FC.append(FFTA[i]*FFTB[i])
        
    # Interpolate
    F = ITERATIVE_INVERSE_FFT(FC)
    for i in range (0,n):
        F[i] = F[i]*(1/n)  
    
    return F

In [14]:
A = [3,2,1,1]
B = [5,0,2,1]
S2 = np.polymul(A,B)
S1 = ITERATIVE_CONVOLUTION(A,B)

In [15]:
np.allclose(S1[0:len(S2)],S2,rtol=1e-05,atol=1e-08,equal_nan=False)

True

In [16]:
for i in range (10):
    A = np.random.random(1024)
    A = A.tolist()
    B = np.random.random(1024)
    B = B.tolist()
    S = np.polymul(A,B)
    SR = RECURSIVE_CONVOLUTION(A,B)
    SI = ITERATIVE_CONVOLUTION(A,B)
    print("Case", i+1, ":")
    print("\t","Recursive Solution:", np.allclose(SR[0:len(S)],S,rtol=1e-05,atol=1e-08,equal_nan=False))
    print("\t","Iterative Solution:", np.allclose(SI[0:len(S)],S,rtol=1e-05,atol=1e-08,equal_nan=False))

Case 1 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 2 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 3 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 4 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 5 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 6 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 7 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 8 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 9 :
	 Recursive Solution: True
	 Iterative Solution: True
Case 10 :
	 Recursive Solution: True
	 Iterative Solution: True


In [17]:
A1 = np.random.random(2048) # 2^11
B1 = np.random.random(2048)
A1 = A1.tolist()
B1 = B1.tolist()
A2 = np.random.random(1024) # 2^10
B2 = np.random.random(1024)
A2 = A2.tolist()
B2 = B2.tolist()
A3 = np.random.random(8192) # 2^13
B3 = np.random.random(8192)
A3 = A3.tolist()
B3 = B3.tolist()

In [18]:
def unitest(A,B):
    S = np.polymul(A,B)
    SR = RECURSIVE_CONVOLUTION(A,B)
    SI = ITERATIVE_CONVOLUTION(A,B)
    print ("Input: \n", "A:", np.asarray(A), "\n", "B:", np.asarray(B))
    print("\t","Recursive Solution:", np.allclose(SR[0:len(S)],S,rtol=1e-05,atol=1e-08,equal_nan=False))
    print("\t","Iterative Solution:", np.allclose(SI[0:len(S)],S,rtol=1e-05,atol=1e-08,equal_nan=False))
    print("\n Output: \n")
    print("Numpy: \n", np.asarray(S))
    print("Recursive: \n", np.asarray(SR))
    print("Iterative: \n", np.asarray(SI))

In [19]:
unitest(A1,B1)

Input: 
 A: [0.74567901 0.42716859 0.83864014 ... 0.         0.         0.        ] 
 B: [0.25796635 0.2640148  0.448177   ... 0.         0.         0.        ]
	 Recursive Solution: True
	 Iterative Solution: True

 Output: 

Numpy: 
 [0.19236009 0.30706542 0.66331594 ... 0.81945899 0.39836784 0.63093124]
Recursive: 
 [1.92360090e-01+2.19485367e-12j 3.07065416e-01+2.37576061e-12j
 6.63315944e-01+2.54853844e-12j ... 3.98367836e-01+5.45494595e-12j
 6.30931243e-01+5.65826654e-12j 1.99833039e-10+4.61609918e-12j]
Iterative: 
 [1.92360090e-01-4.24670975e-12j 3.07065416e-01-4.19110210e-12j
 6.63315944e-01-4.17457674e-12j ... 1.13043463e-11-1.30006762e-11j
 1.13763443e-11-1.30207429e-11j 1.14209843e-11-1.31035742e-11j]


In [20]:
unitest(A2,B2)

Input: 
 A: [0.46312433 0.16717358 0.38295353 ... 0.         0.         0.        ] 
 B: [0.10389584 0.56885436 0.2258708  ... 0.         0.         0.        ]
	 Recursive Solution: True
	 Iterative Solution: True

 Output: 

Numpy: 
 [0.04811669 0.28081894 0.23949096 ... 1.34739651 1.12482685 0.79462368]
Recursive: 
 [ 4.81166907e-02-7.74510664e-13j  2.80818935e-01-8.52423422e-13j
  2.39490961e-01-7.84187912e-13j ...  1.12482685e+00-2.24130722e-12j
  7.94623676e-01-2.14606307e-12j -5.65307801e-11-1.52979166e-12j]
Iterative: 
 [4.81166907e-02-3.54249095e-12j 2.80818935e-01-3.65692016e-12j
 2.39490961e-01-3.53320400e-12j ... 2.00695016e-12-3.44816456e-12j
 1.92795779e-12-3.37623799e-12j 1.98772371e-12-3.49074296e-12j]


In [21]:
unitest(A3,B3)

Input: 
 A: [0.05665022 0.4304864  0.73375027 ... 0.         0.         0.        ] 
 B: [0.25626075 0.50503733 0.04738541 ... 0.         0.         0.        ]
	 Recursive Solution: True
	 Iterative Solution: True

 Output: 

Numpy: 
 [0.01451723 0.13892725 0.4081275  ... 1.05071733 0.37688471 0.40818471]
Recursive: 
 [ 1.45172260e-02-5.13906626e-13j  1.38927244e-01+5.09954973e-13j
  4.08127492e-01+4.74030169e-12j ...  3.76884703e-01-7.69462218e-12j
  4.08184701e-01-9.16294990e-13j -5.23459676e-09+2.41890829e-12j]
Iterative: 
 [ 1.45172284e-02+3.75023448e-10j  1.38927246e-01+3.71747752e-10j
  4.08127495e-01+3.77760109e-10j ... -4.10948636e-10+5.30681301e-10j
 -4.13139939e-10+5.37767841e-10j -4.09338953e-10+5.28422001e-10j]


In [22]:
A4 = np.random.random(32768) # 2^15
B4 = np.random.random(32768)
A4 = A4.tolist()
B4 = B4.tolist()
A5 = np.random.random(131072) # 2^17
B5 = np.random.random(131072)
A5 = A5.tolist()
B5 = B5.tolist()

def unitest2(A,B):
    S = np.polymul(A,B)
    SI = ITERATIVE_CONVOLUTION(A,B)
    print ("Input: \n", "A:", np.asarray(A), "\n", "B:", np.asarray(B))
    print("\t","Iterative Solution:", np.allclose(SI[0:len(S)],S,rtol=1e-05,atol=1e-08,equal_nan=False))
    print("\n Output: \n")
    print("Numpy: \n", np.asarray(S))
    print("Iterative: \n", np.asarray(SI))

In [24]:
unitest2(A4,B4)

Input: 
 A: [0.67190028 0.07042499 0.86666143 ... 0.         0.         0.        ] 
 B: [0.34024541 0.84426884 0.56223703 ... 0.         0.         0.        ]
	 Iterative Solution: True

 Output: 

Numpy: 
 [0.22861099 0.59122625 0.73210242 ... 0.         0.         0.        ]
Iterative: 
 [2.28610989e-01-4.23653132e-09j 5.91226255e-01-4.16360065e-09j
 7.32102423e-01-4.24956610e-09j ... 3.69597537e-11-8.93674448e-10j
 5.58507892e-11-9.62360232e-10j 3.51363355e-11-8.81404123e-10j]


In [25]:
unitest2(A5,B5)

Input: 
 A: [0.61019576 0.89804163 0.8757521  ... 0.         0.         0.        ] 
 B: [0.35731999 0.12891496 0.47110385 ... 0.         0.         0.        ]
	 Iterative Solution: True

 Output: 

Numpy: 
 [0.21803514 0.39955158 0.7161603  ... 0.97705647 0.54336369 0.53004141]
Iterative: 
 [2.18035160e-01+2.08977720e-10j 3.99551603e-01+2.97004785e-11j
 7.16160319e-01+2.13914363e-10j ... 5.43363786e-01-1.44650086e-11j
 5.30041500e-01+1.49538974e-10j 9.06002242e-08-6.99891215e-11j]


In [29]:
from timeit import default_timer as timer

N = 2 
for i in range (15):
    A = np.random.random(N)
    B = np.random.random(N)
    A = A.tolist()
    B = B.tolist()
    
    
    times = []
    for j in range (10):
        start = timer()
        S = ITERATIVE_CONVOLUTION(A,B)
        end = timer()
        time = end-start
        times.append(time)
    
    average = np.mean(times)
    deviation = np.std(times)
    
    print("Size of N", N)
    print("Average Time:", average)
    print("Standard Deviation:", deviation)
    print("\n")
    N*=2

Size of N 2
Average Time: 0.030400244900101826
Standard Deviation: 0.06195259833138895


Size of N 4
Average Time: 0.019209442299779767
Standard Deviation: 0.031180883324244475


Size of N 8
Average Time: 0.04163716730017768
Standard Deviation: 0.06738677297284106


Size of N 16
Average Time: 0.08927487399987513
Standard Deviation: 0.14539602418841532


Size of N 32
Average Time: 0.19235024299987344
Standard Deviation: 0.31183207850056643


Size of N 64
Average Time: 0.401094406899756
Standard Deviation: 0.6389681834724148


Size of N 128
Average Time: 0.8414097208002204
Standard Deviation: 1.3486561014390892


Size of N 256
Average Time: 1.8357663106000472
Standard Deviation: 2.959723901628359


Size of N 512
Average Time: 3.7138247564999802
Standard Deviation: 5.920570433751573


Size of N 1024
Average Time: 7.9705723978999234
Standard Deviation: 12.781655664302042


Size of N 2048
Average Time: 17.189995763000024
Standard Deviation: 27.810203018716045


Size of N 4096
Average Time: 

KeyboardInterrupt: 

In [34]:
def ITERATIVE_CONVOLUTION2(A,B):    
    # Evaluate using FFT
    FFTA = ITERATIVE_FFT(A)
    FFTB = ITERATIVE_FFT(B)
    
    n = len(A)
    
    # Point-Wise Multiplication
    FC = []
    for i in range (0,n):
        FC.append(FFTA[i]*FFTB[i])
        
    # Interpolate
    F = ITERATIVE_INVERSE_FFT(FC)

    return F

In [36]:
N = 2 
for i in range (20):
    A = np.random.random(N)
    B = np.random.random(N)
    A = A.tolist()
    B = B.tolist()
    
    # Double degree bound
    n = len(A)
    for i in range (0,n):
        A.append(0)
        B.append(0)
    
    times = []
    for j in range (10):
        
        start = timer()
        S = ITERATIVE_CONVOLUTION2(A,B)
        end = timer()
        time = end-start
        times.append(time)
    
    average = np.mean(times)
    deviation = np.std(times)
    
    print("Size of N", N)
    print("Average Time:", average)
    print("Standard Deviation:", deviation)
    print("\n")
    N*=2

Size of N 2
Average Time: 0.00041064500001084523
Standard Deviation: 0.0010467618477896581


Size of N 4
Average Time: 0.0001625653000701277
Standard Deviation: 5.869787404954675e-05


Size of N 8
Average Time: 0.0002329482000277494
Standard Deviation: 2.0856667959705398e-05


Size of N 16
Average Time: 0.0005800605000331416
Standard Deviation: 0.00015750598663067378


Size of N 32
Average Time: 0.0010646675002135453
Standard Deviation: 9.834163701536125e-05


Size of N 64
Average Time: 0.0021999719998348154
Standard Deviation: 0.00015993857597299007


Size of N 128
Average Time: 0.004629416200168634
Standard Deviation: 4.259312863119057e-05


Size of N 256
Average Time: 0.010245481400033896
Standard Deviation: 0.0001005938536301305


Size of N 512
Average Time: 0.02230287180009327
Standard Deviation: 0.00031327570333500424


Size of N 1024
Average Time: 0.047172959499766874
Standard Deviation: 0.0007432197061259509


Size of N 2048
Average Time: 0.10659039850006594
Standard Deviation: